In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [2]:

#file_id = '1HOKjjmgU_GAIYA_hZimCD8NTI3yoJ1bL' #Sadegh
file_id = '1wj5EXCBZ9wyzxYoyHk3XHuvV2ZZSTGAb' #Maryam
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('fer2013.csv') 


In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [4]:
dff = df = pd.DataFrame.from_csv('fer2013.csv', index_col=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [5]:
train = df[0:28709]
cv = df[28709:32298]

y_train = train.emotion
y_cv = cv.emotion

train_size = 28709
cv_size = 3589
num_pix = 2304 #48*48
num_classes = 7

In [6]:
X_train = np.zeros((train_size, 48,48,1))
for i in range (train_size):
    b = [int(j) for j in train.pixels[i].split()]
    X_train[i] = np.array(b).reshape([48,48,1])
 


In [7]:
X_cv = np.zeros((cv_size, 48,48,1))
for i in range (28709,32298):
    b = [int(j) for j in cv.pixels[i].split()]
    X_cv[i-28709] = np.array(b).reshape([48,48,1])

In [8]:
import numpy as np
import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [9]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 1, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1,1), padding= 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=1, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 1, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [10]:

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a',padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 1, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f,f), strides = (1,1), name = conv_name_base + '2b',padding = 'same', kernel_initializer= glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 1, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1,1), strides = (1,1), name = conv_name_base + '2c',padding = 'valid', kernel_initializer= glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 1, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1,1), strides = (s,s), name = conv_name_base + '1',padding = 'valid', kernel_initializer= glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 1, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [11]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (48, 48, 1), classes = num_classes):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(32, (5, 5), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [32, 32, 128], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [32, 32, 128], stage=2, block='b')
    X = identity_block(X, 3, [32, 32, 128], stage=2, block='c')
    ### START CODE HERE ##

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='c')
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='d')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='e')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=5, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size = (2,2), name = 'avg_pool')(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [12]:
model = ResNet50()

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Normalize image vectors
X_train = X_train/255.
X_cv = X_cv/255.

# Convert training and test labels to one hot matrices
#y_train = convert_to_one_hot(y_cv, 7).T
#y_cv = convert_to_one_hot(y_cv, 7).T
y_train = keras.utils.to_categorical(y_train, num_classes)
y_cv = keras.utils.to_categorical(y_cv, num_classes)

In [15]:
model.fit(X_train, y_train, epochs = 15, batch_size = 32, validation_data=(X_cv, y_cv))

Train on 28709 samples, validate on 3589 samples
Epoch 1/15
22080/28709 [======================>.......] - ETA: 30s - loss: 2.4478 - acc: 0.2417

28709/28709 [==============================] - 134s 5ms/step - loss: 2.3697 - acc: 0.2427 - val_loss: 1.9344 - val_acc: 0.2469
Epoch 2/15
11808/28709 [===========>..................] - ETA: 1:11 - loss: 1.8182 - acc: 0.2529

28709/28709 [==============================] - 123s 4ms/step - loss: 1.8078 - acc: 0.2511 - val_loss: 1.7957 - val_acc: 0.2480
Epoch 3/15
 7936/28709 [=======>......................] - ETA: 1:27 - loss: 1.8004 - acc: 0.2484

28709/28709 [==============================] - 124s 4ms/step - loss: 1.7928 - acc: 0.2515 - val_loss: 1.7831 - val_acc: 0.2544
Epoch 4/15
 6464/28709 [=====>........................] - ETA: 1:33 - loss: 1.7877 - acc: 0.2506

28709/28709 [==============================] - 123s 4ms/step - loss: 1.7893 - acc: 0.2526 - val_loss: 1.7807 - val_acc: 0.2530
Epoch 5/15
 5888/28709 [=====>........................] - ETA: 1:36 - loss: 1.7870 - acc: 0.2532

...

28709/28709 [==============================] - 123s 4ms/step - loss: 1.4516 - acc: 0.4279 - val_loss: 1.4583 - val_acc: 0.4347
Epoch 12/15
 5536/28709 [====>.........................] - ETA: 1:38 - loss: 1.4330 - acc: 0.4402

28709/28709 [==============================] - 123s 4ms/step - loss: 1.4201 - acc: 0.4453 - val_loss: 1.3843 - val_acc: 0.4653
Epoch 13/15
 5536/28709 [====>.........................] - ETA: 1:37 - loss: 1.3963 - acc: 0.4552

28709/28709 [==============================] - 124s 4ms/step - loss: 1.4088 - acc: 0.4513 - val_loss: 1.3716 - val_acc: 0.4661
Epoch 14/15
 5536/28709 [====>.........................] - ETA: 1:37 - loss: 1.3455 - acc: 0.4780

28709/28709 [==============================] - 124s 4ms/step - loss: 1.3314 - acc: 0.4833 - val_loss: 1.3450 - val_acc: 0.4767
Epoch 15/15
 5536/28709 [====>.........................] - ETA: 1:38 - loss: 1.2964 - acc: 0.4922

28709/28709 [==============================] - 124s 4ms/step - loss: 1.2971 - acc: 0.4989 - val_loss: 1.3410 - val_acc: 0.4762
